 # imports
 

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
stable = True # Set to True for latest pip version or False for main branch in GitHub
!pip install {"tsai -U" if stable else "git+https://github.com/timeseriesAI/tsai.git"} >> /dev/null
from tsai.all import *
computer_setup()

In [ ]:
!pip install wandb
import numpy as np
import matplotlib.pyplot as plt
import math
import torch         
import wandb 
from fastai.callback.wandb import *
import pandas as pd
my_setup(wandb)

In [ ]:
# use when using google drive 
# data is csv 
#data = np.genfromtxt('/content/drive/MyDrive/digitalTwin_data/data_dump_train_groot_dag2V2.csv', delimiter=',')

In [ ]:
df = pd.read_csv('/Users/emile/Downloads/data_dump_train_groot_dag2V2.csv')
data = df.to_numpy()
data.shape

In [ ]:
# min rij 1 en col 1 (index and headers) 
# data = data[1:,:]
# data = data[:,1:]

name = ['time','pos_estimate_main','pos_estimate_rotate','epower','ppower','vel_estimate','pos_setpoint','gyroy','gyrox','gyroz']

In [ ]:
# t to dt 
for i in range(len(data)):
    data[i,0] = data[i,0] - data[i+1,0]

In [ ]:
def get_accuracy(learner):
  dls = learn2.dls
  valid_dl = dls.valid
  valid_probas, valid_targets, valid_preds = learn2.get_preds(dl=valid_dl, with_decoded=True)
  loss = torch.mean(abs(g))
  plt.plot(valid_targets[:])
  plt.plot(valid_preds[:])
  plt.show()
  fig.savefig('model_{},{},{},{}'.format(coloms, window, epochs, lr), bbox_inches='tight', dpi=150)
  
  return loss


def train_get_metrics(coloms, window, epochs, lr,data_len,hz):
  print("Run{}_{}_{}_{}_{}_{}".format(coloms, window, epochs, lr,hz,data_len,))
  df = pd.read_csv('/Users/emile/Downloads/data_dump_train_groot_dag2V2.csv')
  data = df.to_numpy()
  del df
  data = data[:data_len,:]
  X = data[:,coloms]
  Y = data[:,6]
  del data
  # correct window to get constant time samples 
  #window2 = math.floor(window * 1/hz)
  window2 = window
  print(window2)
  # coloms = the colloms for X 
  # window = the window size in samples 
  # epochs, lr are the epochs and leuring rate for training 
  data_len = math.floor(data_len * len(X))
  #if data_len > len(data):
  
  print("length bevore subsampling = ", X.shape)
  # super sample data (trow away every n the value)
  if hz != 1:
    sub = range(0,  len(X), hz)
    X = np.delete(X,sub,0)
  # set dt after subsampling
  if 0 in coloms:
    for i in range(len(X)-1):
      X[i,0] = X[i,0] - X[i+1,0]
    X = X[:-1,:]
  print("length after = ", X.shape)
  # limit X to max time window
  sample_size = math.floor(len(X)/window2)
  X3 = X[0:sample_size*window2,:]
  # x1 = cut data into samples of lengt window time = +-window/100 sec
  # Y1 = labels based on te first position of theplatform
  X1 = np.zeros((sample_size, len(coloms), window2))
  Y1 = np.zeros(sample_size)
  for i in range(sample_size):
    X1[i,:,:] = X3[i*window2:(i+1)*window2,:].T
    Y1[i] = Y[i*window2]
  # split data val and train 
  # 80 20 split
  vel_lim = math.floor(sample_size*0.2)
  X_train = X1[0:sample_size-vel_lim,:,:]
  X_test = X1[sample_size-vel_lim:,:,:]

  y_train = Y1[0:sample_size-vel_lim]
  y_test = Y1[sample_size-vel_lim:]
  
  X, y, splits = combine_split_data([X_train, X_test], [y_train, y_test])
  dsets = TSDatasets(X, y, splits=splits, inplace=True)
  dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[64, 256],batch_tfms=[TSStandardize()],num_workers=0,device=torch.device('cuda'))
  dls.show_batch(sharey=True)
  plt.show()
  for i in range(len(coloms)):
    plt.plot(X1[:,i,1])
    plt.show()
  plt.plot(Y1)


  # create leurner 
  learn = TSRegressor(X, y, splits=splits, bs=[64, 256],batch_tfms=[TSStandardize(by_var=True)],device=torch.device('cuda'))
  #model = InceptionTime(dls.vars, dls.c)

  #learn = Learner(dls, model, metrics=accuracy,loss_func = torch.nn.L1Loss()) 

  if lr == "auto":
    lrfind = learn.lr_find()
    lr = lrfind.valley
    print(lr)
  # fit the model 
  learn.fit_one_cycle(epochs, lr, cbs=[fastai.callback.tracker.EarlyStoppingCallback(monitor='valid_loss', comp=None, min_delta=0.03,patience=15, reset_on_fit=True)])
  learn.export('/Users/emile/Downloads/model_dump/model_{}_{}_{}_{}_{}_{}'.format(coloms, window, epochs, lr,hz,data_len))
  learn.recorder.plot_metrics()
  dls = learn.dls
  valid_dl = dls.valid
  valid_probas, valid_targets, valid_preds = learn.get_preds(dl=valid_dl, with_decoded=True)
  g = valid_targets-valid_preds[:,0]
  loss = torch.sqrt(torch.mean(g**2))
  fig = plt.figure(figsize=(10,5))
  plt.plot(valid_targets[:])
  plt.plot(valid_preds[:])
  # Labeling the X-axis 
  plt.xlabel('time(seconds)') 
  # Labeling the Y-axis 
  plt.ylabel('rotation') 
  plt.legend() 
  print(loss)
  fig.savefig('/Users/emile/Downloads/image_dump/image_{}_{}_{}_{}_{}_{}.jpg'.format(coloms, window, epochs, lr,hz,data_len), bbox_inches='tight', dpi=300)
  X
  return learn, loss

In [ ]:
accelero = [7,8,9]
power = [3] # [3,4]
vel = [5]
time = [0]
coloms = [3,4,7,8,9]
epochs = 30
lr = "auto"
data_len = 2000000

windows = [900,300,600,1200,100] # samples per prediction
lengthes_data = [2000000] # procentile of the data used 
#frequenties = [20, 40, 60, 80, 100] # procentile of sample rate to be used 
frequenties = [1, 2, 3, 4, 5] # trow away one sample every x 
colomsets = [[0,3,5,7,8,9],[0,3,5],[0,3,7,8,9],[0,5,7,8,9]] # sets of colloms leefing one per set

#default 

window = 900
epochs = 50
lr = "auto"
data_len = 1500000
hz = 1


In [ ]:
for data_len in lengthes_data:
  for window in windows:
    for hz in frequenties:
      for coloms in colomsets:
        learn2, loss = train_get_metrics(coloms, window, epochs, lr,data_len,hz)
        print(loss)

In [ ]:
learn2, loss = train_get_metrics([0,3,5],  900, 20, lr,1500000,1)

In [ ]:
losses2 = np.zeros([4,5,20])
for k in range(20):
  j = -1
  for hz in frequenties:
    j = j + 1
    i = -1
    for coloms in colomsets:
      i = i + 1
      learn2, loss = train_get_metrics(coloms, window, epochs, lr,data_len,hz)
      losses2[i,j,k] = loss

In [ ]:
learn2.recorder.plot_metrics()

In [ ]:
print(a.valley)

In [ ]:
#learn2.show_results()
learn2.export()

In [ ]:
dls = learn2.dls
valid_dl = dls.valid
valid_probas, valid_targets, valid_preds = learn2.get_preds(dl=valid_dl, with_decoded=True)
g = valid_targets-valid_preds[:,0]
plt.plot(valid_targets[:1000])
plt.plot(valid_preds[:1000,0])
plt.plot(g[:1000])
plt.show()
print(torch.mean(g**2))

In [ ]:
valid_preds.shape